In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:99%;}
div.CodeMirror {font-family:Consolas; font-size:20pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:19px;}
</style>
"""))

**<font size="5" color="red">ch3 연관분석</font>**
- pip install apyori

# 1. 연관분석 개요
- 데이터들 사이에 자주 발생하는 속성을 찾고, 그 속성들 사이에 연관성이 어느 정도 있는지 분석하는 방법
- 활용분야 : 상품진열, 사기보험적발, 신상품 카테고리 구성....

```
{조건:X ex오렌지주스} -> {결과:Y, ex:와인}
연관분석과 관련 지표
1. 지지도(support) : 전체 데이터 중에 조건결과 항목들의 포함된 거래 비율
     (X, Y)의 항목수 / 전체 데이터수 ex.0.2
2. 신뢰도(confidence) : 조건이 발생했을 때, 결과가 동시에 일어날 확률
     (X, Y)의 항목수 / 조건항목수  ex.0.5
3. 향상도(lift) : 우연히 발생한 규칙은 아닌지 확인
     조건결과 지지도 / (조건지지도)*(결과지지도)
     <1 : 음의 상관관계
     >1 : 양의 상관관계
                           지지도    신뢰도    향상도
{오렌지주스} -> {와인}     1/5(0.2)  1/2(0.5)  0.2/0.4*0.6=0.2/0.24  = 0.833
```

In [4]:
%ls data\cf_basket.csv

 D 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 2D0E-15FE

 D:\ai\source\10_자연어처리\data 디렉터리

2024-12-30  오전 09:03               119 cf_basket.csv
               1개 파일                 119 바이트
               0개 디렉터리  472,865,341,440 바이트 남음


# 2. 연관분석 구현

In [10]:
# 트랜젝션 데이터 가져오기
import csv
transaction = []
with open('data/cf_basket.csv','r', encoding='utf-8') as f:
    csvdata = csv.reader(f)
    #print(list(csvdata))
    for row in csvdata:
        transaction.append(row)
transaction

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['맥주', '콜라', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인']]

In [13]:
from apyori import apriori
rules = apriori(transaction, min_support=0.15, min_confidence=0.1)
rules = list(rules)
len(rules)

18

In [14]:
rules[15]
# items=frozenset({'와인', '콜라', '맥주'}),  support=0.2 : 이 세트가  함께 나타날 확률
# items_base :  조건(lhs)
# items_add : 결과(rhs)

RelationRecord(items=frozenset({'와인', '콜라', '맥주'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'와인', '콜라', '맥주'}), confidence=0.2, lift=1.0), OrderedStatistic(items_base=frozenset({'맥주'}), items_add=frozenset({'와인', '콜라'}), confidence=0.5, lift=1.25), OrderedStatistic(items_base=frozenset({'와인'}), items_add=frozenset({'콜라', '맥주'}), confidence=0.33333333333333337, lift=0.8333333333333334), OrderedStatistic(items_base=frozenset({'콜라'}), items_add=frozenset({'와인', '맥주'}), confidence=0.25, lift=1.25), OrderedStatistic(items_base=frozenset({'와인', '맥주'}), items_add=frozenset({'콜라'}), confidence=1.0, lift=1.25), OrderedStatistic(items_base=frozenset({'콜라', '맥주'}), items_add=frozenset({'와인'}), confidence=0.5, lift=0.8333333333333334), OrderedStatistic(items_base=frozenset({'와인', '콜라'}), items_add=frozenset({'맥주'}), confidence=0.5, lift=1.25)])

In [19]:
row = rules[15]
support = row[1]
ordered_st = row[2]
for item in ordered_st:
    lhs = ', '.join([x for x in item[0]])
    rhs = ', '.join([x for x in item[1]])
    confidence = item[2]
    lift       = item[3]
    if lift>1 : 
        print("{} => {} \t {} \t {} \t {}".format(lhs, rhs, support, confidence, lift))

맥주 => 와인, 콜라 	 0.2 	 0.5 	 1.25
콜라 => 와인, 맥주 	 0.2 	 0.25 	 1.25
와인, 맥주 => 콜라 	 0.2 	 1.0 	 1.25
와인, 콜라 => 맥주 	 0.2 	 0.5 	 1.25


In [28]:
print('조건 -> 결과 \t 지지도    \t신뢰도   \t향상도 ')
for row in rules:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ', '.join([x for x in item[0]])
        rhs = ', '.join([x for x in item[1]])
        confidence = item[2]
        lift       = item[3]
        if lift > 1:
            print("{:10} -> {:10} : {} \t {} \t {}".format(lhs, rhs, 
                                                      support, 
                                                      round(confidence,2), 
                                                      round(lift,2)))

조건 -> 결과 	 지지도    	신뢰도   	향상도 
맥주         -> 콜라         : 0.4 	 1.0 	 1.25
콜라         -> 맥주         : 0.4 	 0.5 	 1.25
소주         -> 콜라         : 0.6 	 1.0 	 1.25
콜라         -> 소주         : 0.6 	 0.75 	 1.25
콜라         -> 소주, 맥주     : 0.2 	 0.25 	 1.25
소주, 맥주     -> 콜라         : 0.2 	 1.0 	 1.25
맥주         -> 와인, 콜라     : 0.2 	 0.5 	 1.25
콜라         -> 와인, 맥주     : 0.2 	 0.25 	 1.25
와인, 맥주     -> 콜라         : 0.2 	 1.0 	 1.25
와인, 콜라     -> 맥주         : 0.2 	 0.5 	 1.25
소주         -> 오렌지주스, 콜라  : 0.2 	 0.33 	 1.67
콜라         -> 오렌지주스, 소주  : 0.2 	 0.25 	 1.25
오렌지주스, 소주  -> 콜라         : 0.2 	 1.0 	 1.25
오렌지주스, 콜라  -> 소주         : 0.2 	 1.0 	 1.67
콜라         -> 와인, 소주     : 0.2 	 0.25 	 1.25
와인, 소주     -> 콜라         : 0.2 	 1.0 	 1.25


In [33]:
import pandas as pd
rules_df = pd.DataFrame(None, columns=['lhs','rhs', 'support', 'confidence', 'lift'])

index = 0
for row in rules:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ', '.join([x for x in item[0]])
        rhs = ', '.join([x for x in item[1]])
        confidence = item[2]
        lift       = item[3]
        if lift > 1:
            rules_df.loc[index] = [lhs, rhs, support, confidence, lift]
            index += 1
rules_df.sort_values(by=['confidence', 'lift'], ascending=False)

,lhs,rhs,support,confidence,lift
13,"오렌지주스, 콜라",소주,0.2,1.000000,1.666667
0,맥주,콜라,0.4,1.000000,1.250000
2,소주,콜라,0.6,1.000000,1.250000
5,"소주, 맥주",콜라,0.2,1.000000,1.250000
8,"와인, 맥주",콜라,0.2,1.000000,1.250000
12,"오렌지주스, 소주",콜라,0.2,1.000000,1.250000
15,"와인, 소주",콜라,0.2,1.000000,1.250000
3,콜라,소주,0.6,0.750000,1.250000
1,콜라,맥주,0.4,0.500000,1.250000
6,맥주,"와인, 콜라",0.2,0.500000,1.250000


# 3. 뉴스 연관 분석
- http://fs.jtbc.joins.com/RSS/economy.xml 기사 검색 후 -> 명사 추출(기자, 앵커 제외) -> 
    ->연관분석 (min_support=0.15, min_confidence=0.1)

In [37]:
(lambda word : word not in ['기자','앵커'])('단군')

True

In [35]:
stopwords = ['기자','앵커']
noun_list = ['홍수', '기자', '앵커','단군']
list(filter(lambda word : word not in stopwords, noun_list))

['홍수', '단군']

In [43]:
import requests
from bs4 import BeautifulSoup
from mecab import MeCab
from apyori import apriori

rss_url = "http://fs.jtbc.joins.com/RSS/economy.xml"
jtbc_economy = requests.get(rss_url)
soup = BeautifulSoup(jtbc_economy.content, 'xml')
link_ele = soup.select('item link')
link_list = [link.text for link in link_ele]

mecab = MeCab()
scripts = []
news = []
stopwords = ['기자','앵커']
for link in link_list:
    response = requests.get(link)
    news_soup = BeautifulSoup(response.content, "html.parser")
    title = news_soup.select_one('title').text.replace('| JTBC 뉴스', '')
    content = str(news_soup.select_one('meta[name="description"]'))
    content = content.replace('<meta content="', '').replace('" name="description"/>','')
    text = title + ' ' + content
    scripts.append(text)
    # postagging 이용한 예
#     noun_list = [word for word, tag in mecab.pos(text) if tag in ('NNG', 'NNP')]
#     noun_list = list(filter(lambda word : word not in stopwords, noun_list))
    # nouns함수를 사용한 예
    # noun_list = list(filter(lambda word : word not in stopwords, mecab.nouns(text)))
    # filter함수 이용 안 한 경우
    noun_list =[word for word, tag in mecab.pos(text) \
                if tag in ('NNG', 'NNP') and word not in stopwords]
    
    news.append(noun_list)
    
rules = apriori(news, min_support=0.15, min_confidence=0.3)
rules = list(rules) # 리스트로 안 바꾸면 generator 객체
import pandas as pd
rules_df = pd.DataFrame(None, 
                columns=['lhs','rhs','support','confidence','lift'])
index = 0
for row in rules:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ','.join(x for x in item[0])
        rhs = ','.join(x for x in item[1])
        confidence = item[2]
        lift      = item[3]
        if lift > 1:
            rules_df.loc[index] = [lhs, rhs, support, confidence, lift]
            index += 1
rules_df

,lhs,rhs,support,confidence,lift
0,가격,이상,0.15,0.75,3.000000
1,이상,가격,0.15,0.60,3.000000
2,가격,최근,0.15,0.75,3.750000
3,최근,가격,0.15,0.75,3.750000
4,말,아람,0.15,0.60,2.400000
5,아람,말,0.15,0.60,2.400000
6,사실,상태,0.15,1.00,6.666667
7,상태,사실,0.15,1.00,6.666667
8,사실,업체,0.15,1.00,6.666667
9,업체,사실,0.15,1.00,6.666667


In [40]:
scripts

["둔촌주공 '전세 홍수' 예상했는데…눈치게임 시작됐다  [앵커]\n\n단군 이래 최대 규모의 재건축이라 불리는 둔촌주공아파트 입주가 다음 달 말부터 시작됩니다. 한꺼번에 대규모 입주가 이뤄지다 보니, 전세매물도 쏟아지지 않을까 했는데 현장 분위기는 다르다고 합니다.\n\n이상화 기자입니다.\n\n[기자]\n\n다음 달 말부터 1만2천 세대가 입주를 시작하는 서울 둔촌동 올림픽파크포레온입니다.\n\n국내 재건축 가운데 역대 최대 규모입니다.\n\n[이용택/공인중개사 : 공개 행사 이후에 찾아오는 분들이 거의 두 배 가량 늘었다고 판단을 하고 있습니다. 다양한 연령층에서 다양한 세대에서 관심을&amp;hellip;]\n\n단지가 워낙 크다 보니 입주 시기 전에 전세 물량이 쏟아져 나올지도 관심입니다.\n\n6년 전, 9천여 세대가 입주한 서울 헬리오시티는 당시 전세뿐 아니라 매매가격도 동반 하락했습니다.\n\n그런데 이번엔 다르다는 게 현장 분위깁니다.\n\n[김복수/공인중개사 : 일반적으로 아파트가 입주하면 30~40% 정도는 전세 나오는 게 보통 시장의 모습인데 여기는 (전세보다) 입주가 많다고 보면 되고, (전세라도) 본인이 받고자 하는 가격을 받고 싶어 하는 거죠]\n\n실거주 의무가 아직 폐지되지 않은 데다 신축 선호 현상도 가세하면서 전세매물 자체가 부족하단 겁니다.\n\n현재 인터넷에 올라온 매물은 3천 세대에도 못 미칩니다.\n\n최근 정부의 대출 관리로 전세대출 규제가 강해진 점도 한몫했습니다.\n\n[권대중/서강대 부동산학과 교수 : 전세대출이 막혔기 때문에 세입자가 물량이 나온다 하더라도 전세대출이 안 되면 들어갈 수가 없거든요. 실입주가 늘어나는 그런 현상 때문에 임대 물량도 없는 것 같습니다.]\n\n상대적으로 저렴하게 전세를 내놨다가 다시 거둬들이는 집주인들도 있는데, 주변 전셋값도 신축을 중심으로 오르는 추세입니다.\n\n[함영진/직방 빅데이터랩장 : 대규모 입주장에 따른 전세 가격 하락이라든지 또 풍부한 매물 유통을 기대하기는 좀 어